# Workspace for Project Euler problems

Here are some things I have learned about jupyter and python that are useful.

## Printing

Print output you want to see by finishing the line with an unassigned value and no semi-colon

In [14]:
myval = 9**8;
myval

43046721

This is nice because when you convert this to a script, that line no longer prints. So you only have to use "print" function when you want the actual converted script to also print that value

### Logging

Use the built-in logging package, like so

In [15]:
import logging, sys
logging.basicConfig(stream=sys.stderr, level=logging.INFO)

log = logging.getLogger("main")

# DEBUG, INFO, WARNING, ERROR, CRITICAL
log.setLevel(logging.INFO)

log.debug('A debug message!')
log.info('We processed %d records', len([1,3,5]))

INFO:main:We processed 3 records
